In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark



In [ ]:
netflix_df=spark.read.options(delimiter=',',header=True, inferSchema=True).csv("/content/netflix_data.csv")
netflix_df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States| 25-Sep-21|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa| 24-Sep-21|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null| 24-Sep-21|        2021| TV-MA| 1 Season|Crime

In [ ]:
netflix_df.columns

['show_id',
 'type',
 'title',
 'director',
 'cast',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description']

In [ ]:
netflix_df.count()

8809

1. How many movies and TV shows are there in the dataset?

In [ ]:
from pyspark.sql.functions import col
num_movies = netflix_df.filter(col('type') == 'Movie').count()
num_tv_shows = netflix_df.filter(col('type') == 'TV Show').count()

print("Number of movies:", num_movies)
print("Number of TV shows:", num_tv_shows)

Number of movies: 6131
Number of TV shows: 2676


2. Who are the top 10 directors with the most titles in the dataset?


In [ ]:
from pyspark.sql.functions import *
#grouped_data = netflix_df.groupBy('director').agg(count('title').alias('count'))
director_counts = netflix_df.groupBy('director').agg(count('title').alias('title_count'))
top_directors = director_counts.filter(director_counts.director.isNotNull()).orderBy(col('title_count').desc())

# Show the top 10 directors with the most titles
top_directors.show(5)

+--------------------+-----------+
|            director|title_count|
+--------------------+-----------+
|       Rajiv Chilaka|         19|
|RaÃºl Campos, Jan...|         18|
|        Marcus Raboy|         16|
|         Suhas Kadav|         16|
|           Jay Karas|         14|
+--------------------+-----------+
only showing top 5 rows



3. What are the top 10 countries with the most content in the dataset?

In [ ]:
country_counts = netflix_df.groupBy('country').agg(count('*').alias('title_count'))
top_countries = country_counts.orderBy(col('title_count').desc())
top_countries.show(10)

+--------------+-----------+
|       country|title_count|
+--------------+-----------+
| United States|       2805|
|         India|        972|
|          null|        832|
|United Kingdom|        419|
|         Japan|        245|
|   South Korea|        199|
|        Canada|        181|
|         Spain|        145|
|        France|        123|
|        Mexico|        110|
+--------------+-----------+
only showing top 10 rows



4. What is the average duration of movies in the dataset?

In [ ]:
sum=netflix_df.filter(col('Movie')).agg(sum('Movie').alias("Total_sum"))

AnalysisException: cannot resolve '`Movie`' given input columns: [cast, country, date_added, description, director, duration, listed_in, rating, release_year, show_id, title, type];
'Filter 'Movie
+- Relation[show_id#16,type#17,title#18,director#19,cast#20,country#21,date_added#22,release_year#23,rating#24,duration#25,listed_in#26,description#27] csv


5. How many titles were added to Netflix each year?

In [ ]:
from pyspark.sql.functions import *
#on basis of relaese year
titles_total=netflix_df.groupBy('release_year').agg(count('*').alias('total_count'))
titles_total.show()
#on basis of release year is 2021
year_2021=titles_total.filter(col('release_year')==2021)
year_2021.show(5)

6. What is the distribution of ratings (G, PG, PG-13, etc.) in the dataset?


In [ ]:
netflix_df.show(3)
ratings_df=netflix_df.groupBy('rating').agg(count('rating').alias("total_count"))
ratings_df.show()

+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States| 25-Sep-21|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa| 24-Sep-21|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null| 24-Sep-21|        2021| TV-MA| 1 Season|Crime

7. How does the distribution of TV shows and movies vary by country?


In [ ]:
netflix_df.show(5)
dis_movies=netflix_df.groupBy('country')


+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States| 25-Sep-21|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa| 24-Sep-21|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null| 24-Sep-21|        2021| TV-MA| 1 Season|Crime

8. What is the average length of movies by country?


What are the most common genres in the dataset?


How many titles were added to Netflix each month?


What are the top 10 longest-running TV shows in the dataset?


Is there a correlation between the release year and the average rating of titles?


How many titles are available in multiple languages?


What are the top 10 actors/actresses with the most appearances in the dataset?


What are the most common words in the descriptions of movies and TV shows?